In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
import psycopg2
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_message_histories import RedisChatMessageHistory

In [3]:

import logging
import os
import tools_restaurant 

In [4]:
load_dotenv()
DB_CONNECTION = os.getenv("DB_CONNECTION")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL=os.getenv("REDIS_URL")
chat = ChatOpenAI(model="gpt-4o", temperature=0)
tools=tools_restaurant.TOOLS

In [5]:
SYSTEM_TEMPLATE = """
Eres un chatbot de asistencia al cliente para el restaurante ROOFTOP magdalena

Tendras 2 funciones principales, una para validar los horarios disponibles dada una fecha y una cantidad de personas, y otra para hacer la reserva
de una mesa. 

1. Obtener los horarios disponibles para hacer reservas dada una fecha y una cantidad de personas
2. Hacer una reserva. Cuando el usuario tenga que hacer una reserva vas a seguir los siguientes pasos :
    a.  le preguntas la fecha y la cantidad de personas. Tomas la fecha y la conviertes al formato YYYY-MM-DD. Si el usuario no te da el año, asume que es 2024.
    b. Validas si esa fecha esta disponible y le muestras los horarios.
    c.  Preguntale por sus datos personales, como el nombre, el telefono y el email. 
        Asegurate que el usuario escriba su nombre y apellido
        Valida que el numero telefonico sea correcto, y agrega el sufijo "+57" al telefono del usuario si es necesario,
        Asegurate que la dirección de correo electronico sea valida
    d. Preguntarle si tiene algun comentario con respecto a la reserva para hacercelo saber al restaurante
    e. Vuelve a mostrarle los datos para confirmar la reserva, no la crees hasta que el usuario confirme.No le puedes dar el id de la reserva

La otra de tus funciones sera responder cualquier duda que tenga el cliente 
respecto al menu del restaurante. Para ello podras usar el contexto que está abajo.
<context>
{context}
</context>
"""

In [6]:
def get_session_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

In [7]:
def create_chain_agent():

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                SYSTEM_TEMPLATE,
            ),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent= create_openai_tools_agent(chat,tools,prompt)
    agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
    #chain=create_stuff_documents_chain(chat,prompt)
    return agent_executor

In [8]:
def get_chat(chain):
    """ Configura la cadena con historial de mensajes. """
    return RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
    )

In [9]:
#Create chain
agent_executor = create_chain_agent()

#Get the session history
conversation_chain=get_chat(agent_executor)

In [12]:
message_body="Como me llamo ?"
wa_id="3227077343"

In [13]:

conversation_chain.invoke({"input": message_body,"context":"",
                            },
                            {"configurable":{"session_id":wa_id}})

Parent run 0087e2e4-e6f4-41cc-b330-b03c781c2265 not found for run 16f57a43-1ed0-48e6-8cbe-6467dd03f299. Treating as a root run.




> Entering new AgentExecutor chain...
Me dijiste que te llamas Pablo. ¿En qué puedo ayudarte hoy, Pablo?

> Finished chain.


{'input': 'Como me llamo ?',
 'context': '',
 'chat_history': [HumanMessage(content='Hola, me llamo Pablo'),
  AIMessage(content='¡Hola, Pablo! ¿En qué puedo ayudarte hoy?')],
 'output': 'Me dijiste que te llamas Pablo. ¿En qué puedo ayudarte hoy, Pablo?'}